<a href="https://colab.research.google.com/github/Junha9/AI/blob/master/Self_Learning_ML_DL/Chapter_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# import statements
import pandas as pd
import numpy as np
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# downlaod wine data
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [3]:
# split train and test data
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# Let's split once again to get the validation set apart from the test set
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
# Let's validate the tree model using the validation set. It looks overfitted a lot.
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [6]:
# We can cross validate which uses different validation set during iteration. default is 5-fold cross validation
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03561187, 0.04226899, 0.07119417, 0.03720236, 0.02590537]), 'score_time': array([0.00884366, 0.01626134, 0.0106554 , 0.00872207, 0.00342536]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
# Let's see the mean value of the test score all the 5 folds
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
# We can specify the splitter when corss validate. default is KFold for regression, StratifiedKFold for classifier
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# We can shuffle and change the number of splits(folds)
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
# Let's execute some grid search to tune the hyperparamters
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [15]:
# fit will automatically grid search and 5-fold cross validate
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
# It will store the model which has the highest accuracy on best_estimator_ property.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [18]:
# It will also store the best hyperparameters
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [19]:
# We can check the cross validation score
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
# Let's check if the reults is same using best_index_ property
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [21]:
# We have checked that the grid search works fine. Now Let's tune more hyperparameters
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
# Let's see the results
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [25]:
# Let's check the mean test score
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [28]:
# We may use various range of parameter. let's use uniform and randint from scipy.
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 4, 9, 0, 7, 1, 4, 0, 7, 1])

In [31]:
# We can increase the number of sample to get equal distribution
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 108, 105,  97, 108, 105,  99,  87, 104,  82]))

In [30]:
# We can use ugen for float
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.91462768, 0.69315718, 0.9646668 , 0.02936483, 0.41585794,
       0.36601799, 0.33107177, 0.02972097, 0.20490199, 0.95488237])

In [35]:
# We set the random value for the params. iteration number will be specified in RandomizedSearchCV class.
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1, 25)
          }

In [36]:
# We can use RandomizedSearchCV to iterate random value for the hyperparameters.
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs = -1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7964e63c8950>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7964e63cbf80>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7964e67de360>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7964e63c9cd0>},
                   random_state=42)

In [37]:
# Let's check the best params.
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [38]:
# Check the best cross validation result
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [39]:
# Let's check the accuracy using the test set of the best model.
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86
